In [179]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd

def load_data(ticker):

    filename = '../data/' + ticker + '_1min_firstratedata.csv'
    df = pd.read_csv(filename)
    df['ticker'] = ticker

    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["date"] = df["timestamp"].dt.date
    df["time"] = df["timestamp"].dt.time
    return df

ticker = 'SPY'
df = load_data(ticker)

In [180]:
# Add the RSI indicator with period n to a df

def RSI(df, n):
    "function to calculate RSI"
    delta = df["close"].diff()
    delta = delta[1:]
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    df["up"] = up.round(4)
    df["down"] = down.round(4)
    AVG_Gain = df["up"].rolling(window=n).mean()
    AVG_Loss = abs(df["down"].rolling(window=n).mean())
    RS = AVG_Gain / AVG_Loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    df["RSI_14"] = RSI.round(4)
    df = df.drop(columns=["up", "down"])

    return df

df = RSI(df, 14)

#df.head()

In [181]:
import datetime
import plotly.graph_objects as go


# Candlestick chart of one day
def chart_range(df, date):
    df = df[df["date"] == date]
    fig = go.Figure(
        data=[
            go.Candlestick(
                x=df["timestamp"],
                open=df["open"],
                high=df["high"],
                low=df["low"],
                close=df["close"],
            )
        ]
    )
    fig.show()


# chart_range(df, datetime.date(2023, 5, 18))


# chart the RSI of one day
def chart_rsi(df, date):
    df = df[df["date"] == date]
    # only chart from 9:30 to 16:00
    df = df[df["time"] >= datetime.time(9, 30, 0)]
    df = df[df["time"] <= datetime.time(16, 0, 0)]
    fig = go.Figure(data=go.Scatter(x=df["timestamp"], y=df["RSI_14"]))
    # add horizontal line at 30 and 70
    fig.add_shape(
        type="line",
        x0=df["timestamp"].min(),
        y0=30,
        x1=df["timestamp"].max(),
        y1=30,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.add_shape(
        type="line",
        x0=df["timestamp"].min(),
        y0=70,
        x1=df["timestamp"].max(),
        y1=70,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.show()


# chart_rsi(df, datetime.date(2023, 5, 18))


def chart_rsi_with_trades(timestamps, rsi_values, enters, exits):
    fig = go.Figure(
        data=go.Scatter(x=timestamps, y=rsi_values, mode="lines", name="RSI")
    )
    fig.add_trace(
        go.Scatter(
            x=enters["timestamps"],
            y=enters["values"],
            mode="markers",
            marker=dict(color="green", size=8),
            name="Enter",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=exits["timestamps"],
            y=exits["values"],
            mode="markers",
            marker=dict(color="red", size=8),
            name="Exit",
        )
    )
    # Add horizontal lines at RSI 30 and 70
    fig.add_shape(
        type="line",
        x0=timestamps[0],
        y0=30,
        x1=timestamps[-1],
        y1=30,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.add_shape(
        type="line",
        x0=timestamps[0],
        y0=70,
        x1=timestamps[-1],
        y1=70,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )

    fig.show()

In [182]:
import datetime
import pandas as pd

def simulate_trade_RSI(date, entry, exit, max_trades, df):
    df_day = df[(df["date"] == date) & (df["time"] >= datetime.time(9, 30, 0)) & (df["time"] <= datetime.time(16, 0, 0))]
    trade_log = pd.DataFrame(columns=["Date", "Direction", "Entry", "Exit", "TimeEnter", "TimeExit", "Gain%", "Gain$"])
    trades, in_trade = 0, False

    for i in range(1, len(df_day)):
        rsi_condition = (df_day["RSI_14"].iloc[i - 1] < entry and df_day["RSI_14"].iloc[i] >= entry)
        exit_condition = (df_day["RSI_14"].iloc[i - 1] > exit and df_day["RSI_14"].iloc[i] <= exit)
        
        if not in_trade and rsi_condition:
            entry_price, in_trade, time_enter = df_day["close"].iloc[i], True, df_day["time"].iloc[i - 1]

        elif in_trade and (exit_condition or i == len(df_day) - 1):
            exit_price = df_day["close"].iloc[i]
            trade_gain_percent = round(((exit_price - entry_price) / entry_price) * 100, 2)
            trade_gain_net = round(exit_price - entry_price, 2)

            trade_log.loc[len(trade_log)] = pd.Series({
                "Date": date,
                "Direction": "Long",
                "Entry": entry_price,
                "Exit": exit_price,
                "TimeEnter": time_enter,
                "TimeExit": df_day["time"].iloc[i],
                "Gain%": trade_gain_percent,
                "Gain$": trade_gain_net,
            })

            trades, entry_price, in_trade = trades + 1, 0, False

            # Check if the maximum number of trades is reached
            if trades >= max_trades:
                break

    return trade_log

In [183]:
# Combine the simulated trades for a range of dates into a single dataframe

def simulate_date_range(start_date, end_date, entry, exit, max_trades, df):
    day_datas = []

    for n in range((end_date - start_date).days + 1):
        date = start_date + datetime.timedelta(n)
        day_data = simulate_trade_RSI(date, entry, exit, max_trades, df)
        if day_data is not None:
            day_datas.append(day_data)

    day_datas = pd.concat(day_datas, ignore_index=True)

    return day_datas

In [184]:
#trades = simulate_date_range(datetime.date(2022, 9, 30), datetime.date(2023, 9, 30), 30, 70, df)

# post-processing of the trades dataframe

def rsi_backtest_explain_results(trades):

    total_gain = trades["Gain$"].sum()
    total_trades = len(trades)

    average_percent_gain = round(trades["Gain%"].mean(), 2)

    print("Total trades: " + str(total_trades))
    print("Average percent gain per trade: " + str(average_percent_gain) + "%")
    print("Total percent gain: " + str(round(total_gain, 2)) + "%")

    # histogram of the percent gain per trade
    import plotly.express as px

    fig = px.histogram(trades, x="Gain%", nbins=100)
    fig.show()

def rsi_ranged_backtest_single_pnl(df, rsi_enter, rsi_exit, maxTrades):
    res = pd.DataFrame()
    res.loc[0, 'StartDate'] = df['Date'].min()
    res.loc[0, 'EndDate'] = df['Date'].max()
    res.loc[0, 'Entry'] = rsi_enter
    res.loc[0, 'Exit'] = rsi_exit,
    res.loc[0, 'MaxTrades'] = maxTrades,
    res.loc[0, 'TotalTrades'] = len(df)
    res.loc[0, 'TotalPercentGain'] = round(df['Gain%'].sum(), 2)
    res.loc[0, 'AveragePercentGain'] = round(df['Gain%'].mean(), 2)
    res.loc[0, 'MaxPercentGain'] = round(df['Gain%'].max(), 2)
    res.loc[0, 'MinPercentGain'] = round(df['Gain%'].min(), 2)

    return res

In [187]:
# # Full stack experiment

# ticker = "SPY"
# max_trades = 2

# df = load_data(ticker)
# df = RSI(df, 14)

# max_trade_options = [1, 2, 3, 4]

# entry_exit_pairs = [
#     (30, 70),
#     (20, 80),
#     (25, 75),
#     (35, 65),
#     (40, 60),
#     (10, 30),
#     (20, 40),
#     (30, 50),
#     (40, 60),
#     (50, 70),
#     (60, 80),
#     (70, 90),
# ]

# results = pd.DataFrame(
#     columns=[
#         "StartDate",
#         "EndDate",
#         "Entry",
#         "Exit",
#         "TotalTrades",
#         "TotalPercentGain",
#         "AveragePercentGain",
#         "MaxPercentGain",
#         "MinPercentGain",
#     ]
# )
# for e in entry_exit_pairs:
#     for f in max_trade_options:
#         trades = simulate_date_range(
#             datetime.date(2022, 9, 30),
#             datetime.date(2023, 9, 30),
#             e[0],
#             e[1],
#             f,
#             df,
#         )
#         pnl = rsi_ranged_backtest_single_pnl(trades, e[0], e[1], f)
#         results.loc[len(results)] = pnl.loc[0]

In [190]:
# # sort by total percent gain
# results = results.sort_values(by=["TotalPercentGain"], ascending=False)
# results

,StartDate,EndDate,Entry,Exit,TotalTrades,TotalPercentGain,AveragePercentGain,MaxPercentGain,MinPercentGain
45,2022-09-30,2023-09-29,70.0,90.0,357.0,15.35,0.04,3.59,-2.18
27,2022-09-30,2023-09-29,20.0,40.0,663.0,11.94,0.02,1.65,-0.88
46,2022-09-30,2023-09-29,70.0,90.0,382.0,11.6,0.03,3.59,-2.91
47,2022-09-30,2023-09-29,70.0,90.0,384.0,11.53,0.03,3.59,-2.91
26,2022-09-30,2023-09-29,20.0,40.0,580.0,10.91,0.02,1.65,-0.88
6,2022-09-30,2023-09-29,20.0,80.0,452.0,8.31,0.02,1.52,-1.94
25,2022-09-30,2023-09-29,20.0,40.0,434.0,8.15,0.02,1.27,-0.88
44,2022-09-30,2023-09-29,70.0,90.0,251.0,8.09,0.03,3.59,-2.18
4,2022-09-30,2023-09-29,20.0,80.0,234.0,7.92,0.03,1.32,-1.63
2,2022-09-30,2023-09-29,30.0,70.0,726.0,7.59,0.01,1.04,-1.7


In [192]:
# Full stack experiment

ticker = "AAPL"
max_trades = 2

df = load_data(ticker)
df = RSI(df, 14)

max_trade_options = [1]

entry_exit_pairs = [(30, 70),(35, 65),(40, 60),(20, 40),(40, 60),(60, 80),(70, 90)]

results = pd.DataFrame(
    columns=[
        "StartDate","EndDate","Entry","Exit","TotalTrades",
        "TotalPercentGain","AveragePercentGain","MaxPercentGain","MinPercentGain"])


for e in entry_exit_pairs:
    for f in max_trade_options:
        print("testing ENTER/EXIT/MAXTRADES:", e[0], e[1], f)
        trades = simulate_date_range(datetime.date(2022, 9, 30), datetime.date(2023, 9, 30), e[0], e[1], f, df)
        pnl = rsi_ranged_backtest_single_pnl(trades, e[0], e[1], f)
        results.loc[len(results)] = pnl.loc[0]

testing 30 70 1
testing 35 65 1
testing 40 60 1
testing 20 40 1
testing 40 60 1
testing 60 80 1
testing 70 90 1


In [194]:
results = results.sort_values(by=["TotalPercentGain"], ascending=False)
results

,StartDate,EndDate,Entry,Exit,TotalTrades,TotalPercentGain,AveragePercentGain,MaxPercentGain,MinPercentGain
6,2022-09-30,2023-09-29,70.0,90.0,251.0,19.84,0.08,4.73,-2.97
5,2022-09-30,2023-09-29,60.0,80.0,251.0,17.59,0.07,2.07,-2.96
2,2022-09-30,2023-09-29,40.0,60.0,251.0,0.29,0.0,1.5,-4.03
4,2022-09-30,2023-09-29,40.0,60.0,251.0,0.29,0.0,1.5,-4.03
3,2022-09-30,2023-09-29,20.0,40.0,239.0,-2.02,-0.01,2.21,-2.64
0,2022-09-30,2023-09-29,30.0,70.0,251.0,-2.75,-0.01,1.61,-3.98
1,2022-09-30,2023-09-29,35.0,65.0,251.0,-4.49,-0.02,1.19,-3.85


So far, APPL and SPY both do best with the 70/90 strat. try more tickers